# 7. BERT 언어모델 기반의 문장 토큰 분류

## 강의 소개

이번 강의에서 소개하는 문장 토큰 분류 모델은 전체 문장에 대한 하나의 라벨이 아닌, 각 토큰에 대한 라벨을 예측합니다. 😀

활용 분야로는 POS-tagging, NER 등이 있습니다. 😁

<br>

## 추가 실습 데이터

- KLUE 프로젝트에서 산출된 POS, DP 데이터셋을 공유합니다 🤗전처리를 통해 학습 데이터 포멧을 맞추고, POS 분류 학습을 시도해봐주세요!
  - [train](https://docs.google.com/spreadsheets/d/1A99uTRh5rKS2xtj4jpTAY3ffhbx70TcEGyKn1LKyxQg/edit?usp=sharing)
  - [val](https://docs.google.com/spreadsheets/d/1pAHy43VNXi5ii8p6ndl8XGQzjeIzts7-eGP1zm17fNk/edit?usp=sharing)

<br>

## Reference

개체명 인식

1. [Named Entity Recognition (NER) for Turkish with BERT](https://medium.com/analytics-vidhya/named-entity-recognition-for-turkish-with-bert-f8ec04a31b0)

QA

1. [lonformer_qa_training.ipynb](https://github.com/patil-suraj/Notebooks/blob/master/longformer_qa_training.ipynb)

2. [[논문리뷰] Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks](https://jeonsworld.github.io/NLP/rag/)

BERT seq2seq

1. [BERT2BERT_for_CNN_Dailymail.ipynb](https://github.com/patrickvonplaten/notebooks/blob/master/BERT2BERT_for_CNN_Dailymail.ipynb)
2. [Bert2Bert Summarization](https://github.com/MrBananaHuman/bert2bert-summarization)

<br>

## 7.1 문장 토큰 관계 분류 task

- 주어진 문장의 각 token이 어떤 범주에 속하는지 분류하는 task

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1UV0GwJFu3n9ABgG5L8MBxPvbMV4EwBvT' width=800/>

<br>

### 7.1.1 Named Entity Recognition (NER)

- 개체명 인식은 문맥을 파악해서 인명, 기관명, 지명 등과 같은 문장 또는 문서에서 특정한 의미를 가지고 있는 단어 또는 어구(개체) 등을 인식하는 과정을 의미한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1Ua2pFV_c2cZM3RxLte-_MSZXX5o1AQoZ' width=800/>

In [ ]:
!pip install pororo

In [5]:
from pororo import Pororo

ner = Pororo(task='ner', lang='ko')

[Korean Sentence Splitter]: Initializing Kss...

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.



In [ ]:
texts = [
    "이수지가 저수지에 갔는데 이 수지가 저수지에 간 걸까 저 수지가 저수지에 간걸까 그 수지가 저수지에 간 걸까 하며",
    "이수지는 고민했는데 고민 끝에 이수의 마이웨이를 부르며 불쾌지수가 올라가며 저수지를 떠나 경기도 수지구의 한 학원으로 달려가더니",
    "지수함수를 배워서 잘 사용하여 주식 수지를 맞아 \"나 이수지, 바로 고단수지! 수지맞았다!\"하며 행복해했다."
]

In [6]:
for text in texts:
    for pred in ner(text):
        if pred[1] != 'O':
            print(f"{pred[0]}({pred[1]})", end="")
        else:
            print(pred[0], end="")
    print()

이수지(PERSON)가 저수지에 갔는데 이 수지가 저수지에 간 걸까 저 수지가 저수지에 간걸까 그 수지가 저수지에 간 걸까 하며
이수지(PERSON)는 고민했는데 고민 끝에 이수(PERSON)의 마이웨이를 부르며 불쾌지수가 올라가며 저수지를 떠나 경기도(LOCATION) 수지구(LOCATION)의 한 학원으로 달려가더니
지수함수를 배워서 잘 사용하여 주식 수지를 맞아 "나 이수지(PERSON), 바로 고단수지! 수지맞았다!"하며 행복해했다.


<br>

### 7.1.2 Part-of-speech tagging (POS TAGGING)

- 품사
  - 단어를 문법적 성질의 공통성에 따라 언어학자들이 몇 갈래로 묶어 놓은 것
- 품사 태깅
  - 주어진 문장의 각 성분에 대하여 가장 알맞는 품사를 태깅하는 것을 의미

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1UijP2__GS9cQwI-O3eoULtGzeLrMCPxe' width=500/>

In [ ]:
!pip install python-mecab-ko

In [12]:
from pororo import Pororo

pos = Pororo(task='pos', lang='ko')
pos('이순신은 조선 중기의 무신이다.')

[('이순신', 'NNP'),
 ('은', 'JX'),
 (' ', 'SPACE'),
 ('조선', 'NNP'),
 (' ', 'SPACE'),
 ('중기', 'NNG'),
 ('의', 'JKG'),
 (' ', 'SPACE'),
 ('무신', 'NNG'),
 ('이', 'VCP'),
 ('다', 'EF'),
 ('.', 'SF')]

<br>

## 7.2 문장 token 분류를 위한 데이터

### 7.2.1 kor_ner

- 한국해양대학교 자연어 처리 연구실에서 공개한 한국어 NER 데이터셋
- 일반적으로, NER 데이터셋은 pos tagging 도 함께 존재

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1Uyeh3SHTZA08Fn0PNKKlV7RUS1uEy6Pf' width=800/>

- Entity tag에서 
  - `B`: Begin, 개체명의 시작(Begin)을 의미
  - `I`: Inner, 내부(Inside)를 의미하며
  - `O`: Out, 다루지 않는 개체명(Outside)를 의미
- ex) `B-PER`
  - 인물명 개체명의 시작을 의미
- ex) `I-PER`
  - 인물명 개체명의 내부 부분을 뜻함

- kor_ner 데이터셋에서 다루는 개체명은 다음과 같다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1UymyyDJVaytgxgBzDd43fnyfZ3tXbUh4' width=800/>

<br>

## 7.3 문장 토큰 분류 모델 학습

### 7.3.1 NER fine-tuning with BERT

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1VDe0MKlfrX7XdFfyocmTjVK4WNdtANjx' width=500/>

<br>

### 7.3.2 주의점

- 형태소 단위의 토큰을 **음절 단위**의 토큰으로 분해하고, Entity tag 역시 음절 단위로 매핑시켜 주어야 한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1VGY2z_w4TqP3qDxVAEsbtKZMTAcD71yL' width=900/>

<br>

### 7.3.3 학습 과정

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1VMIKUeGuZEnmbPVP5AdNEc8xuJnXXEn5' width=800/>